# 第 8 单元第 2 部分：高级深度强化学习。使用 Sample Factory 从像素玩 Doom

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit9/thumbnail2.png" alt="Thumbnail"/>

在本笔记本中，我们将学习如何训练一个深度神经网络，在一个基于游戏 Doom 的 3D 环境中收集物体，下面展示了最终策略的视频。我们使用 [Sample Factory](https://www.samplefactory.dev/)（一种 PPO 算法的异步实现）来训练这个策略。

请注意以下几点：

* [Sample Factory](https://www.samplefactory.dev/) 是一个高级的 RL 框架，**仅在 Linux 和 Mac 上运行**（不支持 Windows）。

* 该框架在**具有多个 CPU 核心的 GPU 机器**上表现最佳，可以达到每秒 10 万次交互的速度。标准 Colab 笔记本上可用的资源**限制了该库的性能**。因此，在此设置下的速度**并不能反映其真实世界的性能**。
* Sample Factory 的基准测试可在多种设置下找到，如果您想了解更多信息，请查看[示例](https://github.com/alex-petrenko/sample-factory/tree/master/sf_examples)。


In [ ]:
from IPython.display import HTML

HTML('''<video width="640" height="480" controls>
  <source src="https://huggingface.co/edbeeching/doom_health_gathering_supreme_3333/resolve/main/replay.mp4"
  type="video/mp4">您的浏览器不支持视频标签。</video>'''
)

为了通过[认证流程](https://huggingface.co/deep-rl-course/en/unit0/introduction#certification-process)验证此实践操作，您需要推送一个模型：

- `doom_health_gathering_supreme` 获得 >= 5 的结果。

要找到您的结果，请前往[排行榜](https://huggingface.co/spaces/huggingface-projects/Deep-Reinforcement-Learning-Leaderboard)并找到您的模型，**结果 = 平均奖励 - 奖励的标准差**

如果您没有找到您的模型，**请到页面底部并点击刷新按钮**

有关认证流程的更多信息，请查看此部分 👉 https://huggingface.co/deep-rl-course/en/unit0/introduction#certification-process

## 设置 GPU 💪
- 为了**加速智能体的训练，我们将使用 GPU**。为此，请转到 `运行时 > 更改运行时类型`

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/gpu-step1.jpg" alt="GPU Step 1">

- `硬件加速器 > GPU`

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/gpu-step2.jpg" alt="GPU Step 2">

在开始训练我们的智能体之前，让我们先**研究一下我们将要使用的库和环境**。

## Sample Factory

[Sample Factory](https://www.samplefactory.dev/) 是**最快的 RL 库之一，专注于策略梯度（PPO）的非常高效的同步和异步实现**。

Sample Factory 经过了**彻底的测试，被许多研究人员和从业者使用**，并得到了积极的维护。我们的实现以**在各种领域达到 SOTA (State-of-the-Art) 性能，同时最大限度地减少 RL 实验的训练时间和硬件要求**而闻名。

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit9/samplefactoryenvs.png" alt="Sample factory"/>



### 主要特点

- 高度优化的算法[架构](https://www.samplefactory.dev/06-architecture/overview/)，以实现最大的学习吞吐量
- [同步和异步](https://www.samplefactory.dev/07-advanced-topics/sync-async/)训练模式
- [串行（单进程）模式](https://www.samplefactory.dev/07-advanced-topics/serial-mode/)，便于调试
- 在基于 CPU 和[GPU 加速的环境](https://www.samplefactory.dev/09-environment-integrations/isaacgym/)中均具有最佳性能
- 单智能体和多智能体训练，自我博弈，支持在一台或多台 GPU 上同时[训练多个策略](https://www.samplefactory.dev/07-advanced-topics/multi-policy-training/)
- 基于种群的训练 ([PBT](https://www.samplefactory.dev/07-advanced-topics/pbt/))
- 离散、连续、混合动作空间
- 基于向量、基于图像、基于字典的观察空间
- 通过解析动作/观察空间规范自动创建模型架构。支持[自定义模型架构](https://www.samplefactory.dev/03-customization/custom-models/)
- 设计为可导入其他项目，[自定义环境](https://www.samplefactory.dev/03-customization/custom-environments/)是一等公民
- 详细的 [WandB 和 Tensorboard 摘要](https://www.samplefactory.dev/05-monitoring/metrics-reference/)，[自定义指标](https://www.samplefactory.dev/05-monitoring/custom-metrics/)
- [HuggingFace 🤗 集成](https://www.samplefactory.dev/10-huggingface/huggingface/)（将训练好的模型和指标上传到 Hub）
- 与多个[示例](https://www.samplefactory.dev/09-environment-integrations/mujoco/) [环境](https://www.samplefactory.dev/09-environment-integrations/atari/) 的[集成](https://www.samplefactory.dev/09-environment-integrations/vizdoom/)，包含[调整好](https://www.samplefactory.dev/09-environment-integrations/dmlab/)的参数和训练好的模型

以上所有策略都可以在 🤗 Hub 上找到。搜索标签 [sample-factory](https://huggingface.co/models?library=sample-factory&sort=downloads)

### Sample Factory 的工作原理

Sample Factory 是**社区可用的最高度优化的 RL 实现之一**。

它的工作原理是**生成多个进程，分别运行 rollout worker、inference worker 和 learner worker**。

这些 *worker* **通过共享内存进行通信，从而降低了进程间的通信成本**。

 *rollout worker* 与环境交互，并将观察结果发送给 *inference worker*。

 *inference worker* 查询一个固定版本的策略，并**将动作发送回 *rollout worker* **。

在 *k* 个步骤之后，rollout worker 将一段经验轨迹发送给 learner worker，**learner worker 用它来更新智能体的策略网络**。

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit9/samplefactory.png" alt="Sample factory"/>

### Sample Factory 中的 Actor-Critic 模型

Sample Factory 中的 Actor-Critic 模型由三个组件构成：

- **编码器 (Encoder)** - 处理输入观察（图像、向量）并将其映射到一个向量。这部分是您最可能需要定制的模型部分。
- **核心 (Core)** - 整合来自一个或多个编码器的向量，在基于记忆的智能体中可以选择性地包含单层或多层的 LSTM/GRU。
- **解码器 (Decoder)** - 在计算策略和价值输出之前，将额外的层应用于模型核心的输出。

该库被设计为自动支持任何观察和动作空间。用户可以轻松添加他们的自定义模型。您可以在[文档](https://www.samplefactory.dev/03-customization/custom-models/#actor-critic-models-in-sample-factory)中找到更多信息。

## ViZDoom

[ViZDoom](https://vizdoom.cs.put.edu.pl/) 是一个**用于 Doom 引擎的开源 Python 接口**。

该库由波兰波兹南理工大学计算科学研究所的 Marek Wydmuch 和 Michal Kempka 于 2016 年创建。

该库支持在多种场景下**直接从屏幕像素训练智能体**，包括团队死亡竞赛（如下方视频所示）。由于 ViZDoom 环境基于 90 年代创建的游戏，它可以在现代硬件上加速运行，**使我们能够相当快速地学习复杂的 AI 行为**。

该库包含以下功能：

- 多平台（Linux、macOS、Windows），
- 提供 Python 和 C++ 的 API，
- [OpenAI Gym](https://www.gymlibrary.dev/) 环境封装器
- 易于创建的自定义场景（提供可视化编辑器、脚本语言和示例），
- 异步和同步的单人及多人模式，
- 轻量级（几 MB）且快速（同步模式下单线程高达 7000 fps），
- 可定制的分辨率和渲染参数，
- 访问深度缓冲区（3D 视觉），
- 自动标记帧内可见的游戏对象，
- 访问音频缓冲区
- 访问参与者/对象列表和地图几何信息，
- 离屏渲染和回合录制，
- 异步模式下的时间缩放。

## 我们首先需要安装 ViZDoom 环境所需的一些依赖项

现在我们的 Colab 运行时已经设置好了，我们可以开始安装在 Linux 上运行 ViZDoom 所需的依赖项。

如果您在自己的 Mac 机器上操作，请遵循 [GitHub 页面](https://github.com/Farama-Foundation/ViZDoom/blob/master/doc/Quickstart.md#-quickstart-for-macos-and-anaconda3-python-36)上的安装说明。

In [ ]:
%%capture
%%bash
# 从以下地址安装 ViZDoom 依赖
# https://github.com/mwydmuch/ViZDoom/blob/master/doc/Building.md#-linux

apt-get install build-essential zlib1g-dev libsdl2-dev libjpeg-dev \
nasm tar libbz2-dev libgtk2.0-dev cmake git libfluidsynth-dev libgme-dev \
libopenal-dev timidity libwildmidi-dev unzip ffmpeg

# Boost 库
apt-get install libboost-all-dev

# Lua 绑定依赖
apt-get install liblua5.1-dev

## 接着我们可以安装 Sample Factory 和 ViZDoom
- 这可能需要 7 分钟

In [ ]:
# 安装 python 库
# 感谢 toinsson
!pip install faster-fifo==1.4.2
!pip install vizdoom

In [ ]:
!pip install sample-factory==2.1.1

## 在 Sample Factory 中设置 Doom 环境

In [ ]:
import functools

from sample_factory.algo.utils.context import global_model_factory
from sample_factory.cfg.arguments import parse_full_cfg, parse_sf_args
from sample_factory.envs.env_utils import register_env
from sample_factory.train import run_rl

from sf_examples.vizdoom.doom.doom_model import make_vizdoom_encoder
from sf_examples.vizdoom.doom.doom_params import add_doom_env_args, doom_override_defaults
from sf_examples.vizdoom.doom.doom_utils import DOOM_ENVS, make_doom_env_from_spec


# 注册所有的 ViZDoom 环境
def register_vizdoom_envs():
    for env_spec in DOOM_ENVS:
        make_env_func = functools.partial(make_doom_env_from_spec, env_spec)
        register_env(env_spec.name, make_env_func)

# Sample Factory 允许注册自定义的神经网络架构
# 更多细节请参阅 https://github.com/alex-petrenko/sample-factory/blob/master/sf_examples/vizdoom/doom/doom_model.py
def register_vizdoom_models():
    global_model_factory().register_encoder_factory(make_vizdoom_encoder)


def register_vizdoom_components():
    register_vizdoom_envs()
    register_vizdoom_models()

# 解析命令行参数并创建配置
def parse_vizdoom_cfg(argv=None, evaluation=False):
    parser, _ = parse_sf_args(argv=argv, evaluation=evaluation)
    # Doom 环境特有的参数
    add_doom_env_args(parser)
    # 覆盖算法参数的 Doom 默认值
    doom_override_defaults(parser)
    # 第二遍解析产生最终配置
    final_cfg = parse_full_cfg(parser, argv)
    return final_cfg

现在设置已经完成，我们可以开始训练智能体了。我们在这里选择学习一个名为 `Health Gathering Supreme` 的 ViZDoom 任务。

### 场景：Health Gathering Supreme

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit9/Health-Gathering-Supreme.png" alt="Health-Gathering-Supreme"/>



这个场景的目标是**教会智能体如何在不知道是什么让他生存下去的情况下生存**。智能体只知道**生命是宝贵的**，死亡是坏事，所以**它必须学会什么能延长它的存在，以及它的健康与此有关**。

地图是一个包含墙壁的矩形，地板是绿色的酸性地面，会**周期性地对玩家造成伤害**。最初，地图上均匀分布着一些医疗包。每隔一段时间会有一个新的医疗包从天而降。**医疗包可以恢复玩家的部分生命值**——为了生存，智能体需要捡起它们。当玩家死亡或超时后，本回合结束。

更多配置：
- 生存奖励 = 1
- 3 个可用按钮：左转，右转，前进
- 1 个可用游戏变量：HEALTH (生命值)
- 死亡惩罚 = 100

您可以在[这里](https://github.com/Farama-Foundation/ViZDoom/tree/master/scenarios)找到更多关于 ViZDoom 中可用场景的信息。

还有一些为 ViZDoom 创建的更复杂的场景，例如在[这个 GitHub 页面](https://github.com/edbeeching/3d_control_deep_rl)上详细介绍的那些。



## 训练智能体
- 我们将训练智能体 4,000,000 步，大约需要 20 分钟

In [ ]:
## 开始训练，这大约需要 15 分钟
register_vizdoom_components()

# 我们今天训练的场景是 health gathering
# 其他场景包括 "doom_basic", "doom_two_colors_easy", "doom_dm", "doom_dwango5", "doom_my_way_home", "doom_deadly_corridor", "doom_defend_the_center", "doom_defend_the_line"
env = "doom_health_gathering_supreme"
cfg = parse_vizdoom_cfg(argv=[f"--env={env}", "--num_workers=8", "--num_envs_per_worker=4", "--train_for_env_steps=4000000"])

status = run_rl(cfg)

## 让我们看看训练好的策略的表现，并输出一个智能体的视频。

In [ ]:
from sample_factory.enjoy import enjoy
cfg = parse_vizdoom_cfg(argv=[f"--env={env}", "--num_workers=1", "--save_video", "--no_render", "--max_num_episodes=10"], evaluation=True)
status = enjoy(cfg)

## 现在让我们可视化智能体的表现

In [ ]:
from base64 import b64encode
from IPython.display import HTML

mp4 = open('/content/train_dir/default_experiment/replay.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=640 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

智能体学到了一些东西，但它的表现还可以更好。我们显然需要训练更长的时间。不过，让我们先把这个模型上传到 Hub。

## 现在让我们将您的检查点和视频上传到 Hugging Face Hub




为了能够与社区分享您的模型，还需要遵循以下三个步骤：

1️⃣ (如果还没有) 创建一个 HF 账户 ➡ https://huggingface.co/join

2️⃣ 登录，然后您需要存储来自 Hugging Face 网站的身份验证令牌。
- 创建一个新令牌 (https://huggingface.co/settings/tokens)，**角色需为 write (写入)**

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/create-token.jpg" alt="Create HF Token">

- 复制令牌
- 运行下面的单元格并粘贴令牌

如果您不想使用 Google Colab 或 Jupyter Notebook，您需要改用此命令：`huggingface-cli login`

In [ ]:
from huggingface_hub import notebook_login
notebook_login()
!git config --global credential.helper store

In [ ]:
from sample_factory.enjoy import enjoy

hf_username = "ThomasSimonini" # 在此处插入您的 HuggingFace 用户名

cfg = parse_vizdoom_cfg(argv=[f"--env={env}", "--num_workers=1", "--save_video", "--no_render", "--max_num_episodes=10", "--max_num_frames=100000", "--push_to_hub", f"--hf_repository={hf_username}/rl_course_vizdoom_health_gathering_supreme"], evaluation=True)
status = enjoy(cfg)

## 让我们加载另一个模型




这个智能体的表现不错，但还可以更好！让我们从 Hub 下载并可视化一个训练了 100 亿个时间步的智能体。

In [ ]:
# 从 hub 下载智能体
!python -m sample_factory.huggingface.load_from_hub -r edbeeching/doom_health_gathering_supreme_2222 -d ./train_dir


In [ ]:
!ls train_dir/doom_health_gathering_supreme_2222

In [ ]:
env = "doom_health_gathering_supreme"
cfg = parse_vizdoom_cfg(argv=[f"--env={env}", "--num_workers=1", "--save_video", "--no_render", "--max_num_episodes=10", "--experiment=doom_health_gathering_supreme_2222", "--train_dir=train_dir"], evaluation=True)
status = enjoy(cfg)

In [ ]:
mp4 = open('/content/train_dir/doom_health_gathering_supreme_2222/replay.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=640 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

## 一些额外挑战 🏆：Doom 死亡竞赛

训练一个能玩 Doom 死亡竞赛的智能体**需要在一台比 Colab 中可用机器配置更强的机器上花费数小时**。

幸运的是，我们**已经在这个场景中训练好了一个智能体，并且它可以在 🤗 Hub 上找到！** 让我们下载模型并可视化智能体的表现。

In [ ]:
# 从 hub 下载智能体
!python -m sample_factory.huggingface.load_from_hub -r edbeeching/doom_deathmatch_bots_2222 -d ./train_dir

由于智能体游戏时间很长，视频生成可能需要**10分钟**。

In [ ]:

from sample_factory.enjoy import enjoy
register_vizdoom_components()
env = "doom_deathmatch_bots"
cfg = parse_vizdoom_cfg(argv=[f"--env={env}", "--num_workers=1", "--save_video", "--no_render", "--max_num_episodes=1", "--experiment=doom_deathmatch_bots_2222", "--train_dir=train_dir"], evaluation=True)
status = enjoy(cfg)
mp4 = open('/content/train_dir/doom_deathmatch_bots_2222/replay.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=640 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)


您**可以尝试使用上面的代码在这个环境中训练您的智能体**，但不能在 Colab 上进行。
**祝您好运 🤞**

如果您喜欢更简单的场景，**为什么不尝试在另一个 ViZDoom 场景中进行训练，例如 `doom_deadly_corridor` 或 `doom_defend_the_center`。**



---


最后一个单元到此结束。但我们还没有完！ 🤗 接下来的**附加部分将包含深度强化学习中一些最有趣、最前沿和最尖端的工作**。

## 保持学习，保持出色 🤗